In [7]:
from pyspark.sql.session import SparkSession
from pyspark import SparkContext
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [8]:
aa =  open('venu.txt','w')
vv = open('/home/venugoapal/gopal.txt','r')

In [9]:
aa =  open('venu.txt','w')
vv = open('/home/venugoapal/data2.txt','r')
#!/usr/bin/env python

from operator import itemgetter
import sys

current_word = None
current_count = ''
word = None

# read the entire line from STDIN
for line in vv:
    if "#" in line:
         continue
    # remove leading and trailing whitespace
    line = line.strip().split('\t')
    # slpiting the data on the basis of tab we have provided in mapper.py
    word = line[0]
    try:
         cc=line[1]
    except:
          cc = ' '
    if cc == ' ':
        pass   
    else:
         cc  =  cc +','+str((int(word)+int(cc))%20+1)+':'
    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count +=cc
    else:
        if current_word=='0':
            aa.write('{}\t{}\t{}'.format(current_word,0, current_count)) 
            aa.write('\n')
        elif current_word:
            # write result to STDOUT
            aa.write('{}\t{}\t{}'.format(current_word,999, current_count))
            aa.write('\n')
        current_count = cc
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    aa.write('{}\t{}\t{}'.format(current_word,999, current_count))  
aa.close()  

In [10]:
textFile = sc.textFile("/home/venugoapal/venu.txt")
count = sc.accumulator(0)

def customSplitNodesTextFile(node):
	if len(node.split('\t')) < 3:
		nid, distance = node.split('\t')
		neighbors = None
	else:
		nid, distance, neighbors = node.split('\t')
		neighbors = neighbors.split(':')
		neighbors = neighbors[:len(neighbors) - 1]
	path = nid
	return (nid , (int(distance), neighbors, path))

def customSplitNodesIterative(node):
	nid = node[0]
	distance = node[1][0]
	neighbors = node[1][1]
	path = node[1][2]
	elements = path.split('->')
	if elements[len(elements) - 1] != nid:
		path = path + '->' + nid;
	return (nid , (int(distance), neighbors, path))

def customSplitNeighbor(parentPath, parentDistance, neighbor):
	if neighbor!=None:
		nid, distance = neighbor.split(',')
		distance = parentDistance + int(distance)
		path = parentPath + '->' + nid
		return (nid, (int(distance), 'None', path))

def minDistance(nodeValue1, nodeValue2):
	neighbors = None
	distance = 0
	path = ''
	if nodeValue1[1] != 'None':
		neighbors = nodeValue1[1]
	else:
		neighbors = nodeValue2[1]
	dist1 = nodeValue1[0]
	dist2 = nodeValue2[0]
	if dist1 <= dist2:
		distance = dist1
		path = nodeValue1[2]
	else:
		count.add(1);
		distance = dist2
		path = nodeValue2[2]
	return (distance, neighbors, path)

def formatResult(node):
	nid = node[0]
	minDistance = node[1][0]
	path = node[1][2]
	return nid, minDistance, path

nodes = textFile.map(lambda node: customSplitNodesTextFile(node))

oldCount = 0
iterations = 0
while True:
	iterations += 1
	nodesValues = nodes.map(lambda x: x[1])
	neighbors = nodesValues.filter(lambda nodeDataFilter: nodeDataFilter[1]!=None).map(
		lambda nodeData: map(
			lambda neighbor: customSplitNeighbor(
				nodeData[2], nodeData[0], neighbor
			), nodeData[1]
		)
	).flatMap(lambda x: x)
	mapper = nodes.union(neighbors)
	reducer = mapper.reduceByKey(lambda x, y: minDistance(x, y))
	nodes = reducer.map(lambda node: customSplitNodesIterative(node))
	nodes.count() # We call the count to execute all the RDD transformations
	if oldCount == count.value:
		break
	oldCount=count.value
print('Finished after: ' + str(iterations) + ' iterations')
result = reducer.map(lambda node: formatResult(node))

result.collect()

Finished after: 9 iterations


[('18', 86, '0->5->6->7->12->21->18'),
 ('17', 70, '0->5->4->9->14->15->16->17'),
 ('12', 52, '0->5->6->7->12'),
 ('15', 44, '0->5->4->9->14->15'),
 ('16', 56, '0->5->4->9->14->15->16'),
 ('6', 18, '0->5->6'),
 ('5', 6, '0->5'),
 ('19', 32, '0->5->10->19'),
 ('20', 52, '0->5->10->19->20'),
 ('14', 34, '0->5->4->9->14'),
 ('8', 60, '0->5->6->7->12->13->8'),
 ('13', 58, '0->5->6->7->12->13'),
 ('0', 0, '0'),
 ('3', 43, '0->5->6->7->3'),
 ('7', 32, '0->5->6->7'),
 ('9', 30, '0->5->4->9'),
 ('21', 66, '0->5->6->7->12->21'),
 ('10', 22, '0->5->10'),
 ('11', 24, '0->5->10->11'),
 ('4', 16, '0->5->4'),
 ('2', 27, '0->5->6->2')]